In [74]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import os
from numpy.linalg import norm

In [3]:
data_EHR1 = pd.read_csv("./data_EHR1.csv")
data_EHR1.head()

,f1_1,f2_1,f3_1,f4_1,f5_1,f6_1,f7_1,f8_1,f9_1,f10_1,f11_1,f12_1,f13_1,f14_1,f15_1,f16_1,f17_1,f18_1,f19_1,f20_1
0,1.696794,-0.621645,-2.350322,0.857568,0.183480,0.929362,1.662710,3.002824,-0.952821,0.780778,-2.149981,-0.096125,0.686564,0.932175,1.035598,0.100794,0.472664,0.961984,-1.581007,0.418345
1,4.490176,-1.369651,-6.381362,2.446793,0.648104,5.127069,3.632748,6.645584,-1.694836,1.961207,-5.393690,-1.725572,1.115977,2.599284,1.849126,0.149453,1.651858,2.380088,-2.989520,1.523643
2,6.195791,-1.803062,-8.885239,1.678346,1.484633,7.669556,5.218378,9.897765,-2.370407,2.332558,-7.979431,-2.123378,1.262524,3.922189,2.477547,1.246901,1.795737,2.639566,-4.948484,1.801274
3,5.943051,0.117324,-11.007327,0.045042,2.026570,7.370177,6.339163,11.226333,-2.778230,3.804610,-8.785526,-1.515712,0.418063,2.784484,2.132019,1.920771,1.803917,2.866873,-4.924506,1.716181
4,1.792690,2.802358,-5.212722,-1.270905,0.897933,4.450223,2.796456,7.543190,-1.245164,1.265860,-4.883092,-0.436753,-0.691122,2.223674,-0.473904,0.735597,1.970611,0.547733,-3.001847,0.299120


In [7]:
data_EHR1.shape

(60000, 20)

In [4]:
data_EHR2 = pd.read_csv("./data_EHR2.csv")
data_EHR2.head()

,f1_2,f2_2,f3_2,f4_2,f5_2,f15_2,f14_2,f6_2,f17_2,f9_2,f7_2,f12_2,f20_2,f19_2,f11_2,f18_2,f10_2,f16_2,f8_2,f13_2
0,1.002464,-0.054770,-1.607723,0.413153,0.526490,0.808553,0.521133,1.489252,1.093870,-0.143866,1.521181,-1.214374,-0.054487,-0.620618,-0.165420,0.877329,0.684882,-0.041809,2.285389,-0.688339
1,-0.208255,-0.430312,3.542310,0.273496,0.523185,1.403259,0.807507,-0.765867,0.872294,0.744647,-0.662414,0.211199,-0.622928,0.033095,1.369378,1.721680,-1.527514,-0.234272,0.200160,-0.147486
2,-3.177253,-0.359707,7.928680,0.673136,-0.539660,0.083743,0.467750,-3.756303,-0.516528,1.122243,-4.087978,-0.085167,-1.435108,-0.203887,3.237390,1.130163,-3.310383,-1.228359,-3.529958,0.776517
3,-3.293687,-0.708697,6.471826,0.595019,-2.137115,-0.089151,-1.102489,-3.336020,-0.365526,0.826688,-4.684949,-0.328677,-1.541318,-0.260947,2.497054,-0.693162,-2.074722,-1.027206,-3.263632,0.990452
4,0.941395,-1.305723,3.126918,-0.427142,-2.480852,-0.253852,-1.456013,-1.420847,0.261801,0.014554,-1.940401,-0.648343,-1.986704,-0.346978,-0.000081,-1.558701,-0.040872,-0.748870,-0.606107,1.572000


In [8]:
data_EHR2.shape

(60000, 20)

# Get golden-standard-list from permutation matrix for the unmapped features in 2 EHRs

In [5]:
# get permutation matrix for the unmapped features in 2 EHRs
ump_f_EHR1 = list(data_EHR1.columns[5:])
ump_f_EHR2 = list(data_EHR2.columns[5:])

print(ump_f_EHR1)
print(ump_f_EHR2)

['f6_1', 'f7_1', 'f8_1', 'f9_1', 'f10_1', 'f11_1', 'f12_1', 'f13_1', 'f14_1', 'f15_1', 'f16_1', 'f17_1', 'f18_1', 'f19_1', 'f20_1']
['f15_2', 'f14_2', 'f6_2', 'f17_2', 'f9_2', 'f7_2', 'f12_2', 'f20_2', 'f19_2', 'f11_2', 'f18_2', 'f10_2', 'f16_2', 'f8_2', 'f13_2']


In [6]:
p = np.zeros((len(ump_f_EHR1), len(ump_f_EHR2)))
for i in range(len(ump_f_EHR1)):
    for j in range(len(ump_f_EHR2)):
        if ump_f_EHR1[i][1:-2] == ump_f_EHR2[j][1:-2]:
            p[i][j] = 1
p

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.

# Training seq-to-seq model to regard transformation function as the fingerprint

In [11]:
# transfer gen_data shape from (60000, 20) to (2500, 24, 20)
data_1 = data_EHR1.to_numpy().reshape((2500, 24, 20))
data_2 = data_EHR2.to_numpy().reshape((2500, 24, 20))

### Define Dataset
* input: unmapped feature i in 2 EHRS (totally 15 unmapped features in 2 EHRs, the last 15 columns in data_1/data_2)
* target: pre-mapped features in 2 EHRS (the first 5 columns)


In [39]:
class Seq2seq_Dataset(Dataset):
    def __init__(self, data, EHR_version): 
        self.patients_num = data.shape[0]
        self.ump_features_list = []
        self.mp_features_list = []
        
        # load mp_features
        if os.path.exists("./mp_f_tensors_" + str(EHR_version) + ".pt"):
            self.mp_features_list = torch.load("./mp_f_tensors_" + str(EHR_version) + ".pt")
            print("Finish loading mapped features' tensors!")
        else:
            for i in range(self.patients_num):
                cur_mp_features = torch.tensor(data[i,:,:5])
                self.mp_features_list.append(cur_mp_features.float())
            
            torch.save(self.mp_features_list, "./mp_f_tensors_" + str(EHR_version) + ".pt")
            print("Finish transforming mapped features' tensors!")
        
        # load ump_features
        if os.path.exists("./ump_f_tensors_" + str(EHR_version) + ".pt"):
            self.ump_features_list = torch.load("./ump_f_tensors_" + str(EHR_version) + ".pt")
            print("Finish loading unmapped features' tensors!")
        else:
            for i in range(self.patients_num):
                cur_ump_features = torch.tensor(data[i, :, 5:])
                self.ump_features_list.append(cur_ump_features.float())
                
            torch.save(self.ump_features_list, "./ump_f_tensors_" + str(EHR_version) + ".pt")
            print("Finish transforming unmapped features' tensors!")
            
    def __len__(self):
        return self.patients_num
    
    def __getitem__(self, idx):
        sample_mp_features = self.mp_features_list[idx]
        # shape: [24, 5]
        sample_ump_features = self.ump_features_list[idx]
        # shape: [24, 15]
        return sample_mp_features, sample_ump_features
        
        

# Define model 

In [15]:
import torch.nn as nn
from torch import optim
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import average_precision_score, roc_auc_score
import random
# from torch.nn import BCEWithLogitsLoss

In [37]:
class Seq2seq_model(nn.Module):
    def __init__(self, ump_feature_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f):
        super(Seq2seq_model, self).__init__()
        # each model trained on only one unmapped feature: ump_feature_id
        # currently, ump_feature_id takes value from [0, 14]
        self.ump_id = ump_feature_id
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.h_0 = torch.randn(1, self.batch_size, hidden_dim)
        # RNN to get final hidden states
        self.RNN = nn.RNN(input_dim, hidden_dim, batch_first=True, dropout=0.2)
        # MLP to predict pre-mapped features(flatten to shape: seq_len x num_mp_f)
        self.target_dim = seq_len * num_mp_f
        self.dense1 = nn.Linear(hidden_dim, hidden_dim)
        self.MLP_drop1 = nn.Dropout(p=0.2)
        self.dense2 = nn.Linear(hidden_dim, int(self.target_dim / 2))
        self.MLP_drop2 = nn.Dropout(p=0.2)
        self.dense3 = nn.Linear(int(self.target_dim / 2), self.target_dim)
        self.MLP_drop3 = nn.Dropout(p=0.2)
        self.dense4 = nn.Linear(self.target_dim, self.target_dim)
        
    def forward(self, input_data):
        true_mapped_features = nn.Flatten()(input_data[0]) 
        # shape: [10, 24*5], targets
        unmapped_features = input_data[1][:, :, self.ump_id].reshape((self.batch_size, self.seq_len, 1)) 
        # shape: [10, 24, 1], includes all unmapped features
        output, h_n = self.RNN(unmapped_features, self.h_0)
        # h_n: final hidden state with shape [1, batch_size, hidden_dim]
        map_predict = self.dense1(h_n[0])
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop1(map_predict)
        
        map_predict = self.dense2(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop2(map_predict)
        
        map_predict = self.dense3(map_predict)
        map_predict = nn.ReLU()(map_predict)
        map_predict = self.MLP_drop3(map_predict)
        
        map_predict = self.dense4(map_predict) # shape [batch_size, seq_len * num_mp_f]
        
        criterion = nn.MSELoss()
        loss = criterion(map_predict, true_mapped_features)
        return {"predicts": map_predict, "loss": loss}
        
        
        

# K-Fold Cross Validation

### a. Define train(), val() to call in each epoch

In [89]:
def train(train_loader, model, optimizer, batch_size):
    size = len(train_loader.dataset)
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        cur_loss = model(data)["loss"]
        cur_loss.backward()
        optimizer.step()
#         if i % 100 == 0:
#             loss, current = cur_loss.item(), i*len(data)
#             print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
        
            

def val(val_loader, model, batch_size):
    val_loss_sum = 0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_loss_sum += model(data)["loss"]
    avg_val_loss = val_loss_sum / len(val_loader)
    return avg_val_loss
            

### b. Split dataset into train+val (90%) & final_test (10%) parts 

In [40]:
from torch.utils.data import random_split, DataLoader


dataset_1_all = Seq2seq_Dataset(data_1, 1)
dataset_2_all = Seq2seq_Dataset(data_2, 2)

dataset_1_kfold, dataset_1_final_test = random_split(dataset_1_all, [int(len(dataset_1_all) * 0.9), int(len(dataset_1_all) * 0.1)], generator=torch.Generator().manual_seed(42))
dataset_2_kfold, dataset_2_final_test = random_split(dataset_2_all, [int(len(dataset_2_all) * 0.9), int(len(dataset_2_all) * 0.1)], generator=torch.Generator().manual_seed(42))


Finish transforming mapped features' tensors!
Finish transforming unmapped features' tensors!
Finish transforming mapped features' tensors!
Finish transforming unmapped features' tensors!


### c. K-Fold cross validation to train and val model

In [44]:
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
from torch import optim

num_epochs = 10
batch_size = 10
input_dim = 1 # each unmapped feature
hidden_dim = 20 # hidden dim of RNN model
num_ump_f = 15
seq_len = 24
num_mp_f = 5
lr = 0.002
k_folds = 5
splits=KFold(n_splits=k_folds, shuffle=True,random_state=42)
kfold_test_loss_1 = np.zeros((num_ump_f, k_folds))
kfold_test_loss_2 = np.zeros((num_ump_f, k_folds))

# trained on unmapped features in EHR1
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_1_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_1_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = Seq2seq_model(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_1[ump_f_id, fold] = cur_sum_loss / num_epochs
        
# trained on unmapped features in EHR2
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset_2_kfold)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset_2_kfold, batch_size=batch_size, sampler=test_sampler)
    for ump_f_id in range(num_ump_f):
        model = Seq2seq_model(ump_f_id, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
        optimizer = optim.Adam(model.parameters(), lr = lr)
        cur_sum_loss = 0
        for epoch in range(num_epochs):
            train(train_loader, model, optimizer, batch_size)
            cur_sum_loss += val(test_loader, model, batch_size)
            
        kfold_test_loss_2[ump_f_id, fold] = cur_sum_loss / num_epochs
    

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


In [45]:
kfold_test_loss_1

array([[8.18213463, 8.04709721, 8.00663662, 8.06370831, 8.20800972],
       [7.66380835, 7.60657501, 7.65536594, 7.49697638, 7.75573874],
       [8.00084209, 7.98895025, 7.9028368 , 7.89055109, 8.01455307],
       [9.64537334, 9.24072456, 9.45179558, 9.28697777, 9.58749962],
       [8.25765228, 8.00774956, 8.08473015, 7.97795248, 8.16930199],
       [8.38163567, 8.17290974, 8.04883099, 7.86730051, 8.11237717],
       [9.47379303, 9.24890041, 9.22126675, 9.24496269, 9.29763222],
       [9.94843674, 9.64812946, 9.83619881, 9.62560272, 9.88210869],
       [9.93941498, 9.62500858, 9.8076458 , 9.6150341 , 9.84286118],
       [9.36569118, 9.07037258, 9.23637295, 8.99040127, 9.33036423],
       [9.931633  , 9.62621975, 9.80823326, 9.6090517 , 9.84626579],
       [9.8164959 , 9.56373024, 9.76970959, 9.50335884, 9.76703835],
       [9.01693058, 8.68861008, 8.77471256, 8.67355728, 8.83125019],
       [9.30682373, 9.12713432, 9.21259117, 9.01767159, 9.37334633],
       [9.62224197, 9.30284023, 9.

In [46]:
kfold_test_loss_2

array([[ 9.0212326 ,  9.18507957,  9.13533592,  9.18954945,  9.76801109],
       [ 9.6192503 ,  9.88075733,  9.76010895,  9.67849541, 10.35847187],
       [ 7.92218494,  8.06856155,  8.06936646,  7.91815281,  8.7112627 ],
       [ 9.52457237,  9.75982857,  9.64916706,  9.63474655, 10.26062298],
       [ 9.23663139,  9.48885059,  9.33452034,  9.35671711,  9.94048119],
       [ 7.22630835,  7.42509317,  7.57019329,  7.65055466,  8.04555988],
       [ 9.13063145,  9.34042358,  9.25703239,  9.15375996,  9.76223183],
       [ 9.36997318,  9.58489799,  9.44891548,  9.46282005, 10.12315369],
       [ 8.9656601 ,  9.25315762,  9.19209766,  9.10543823,  9.81880379],
       [ 7.80669403,  8.04145527,  8.2769537 ,  8.08746624,  8.60482979],
       [ 8.50068474,  8.86130333,  8.79769611,  8.63900948,  9.24771118],
       [ 7.90849543,  8.09234524,  8.11461067,  8.15523624,  8.60020161],
       [ 9.64540577,  9.90639877,  9.75844288,  9.69396019, 10.37814522],
       [ 7.78965092,  7.90408421,  8.0

### d. Trained the final model using all the 90% training data (data used in the above 5-fold cross-validation process)

In [90]:
final_train_dataloader_1 = DataLoader(dataset_1_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = Seq2seq_model(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_1, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./each_umpf_model_param/EHR1/umpf_"+str(umpf_id+1)+"_EHR1.pth")

current unmapped feature id:  0


/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


In [91]:
final_train_dataloader_2 = DataLoader(dataset_2_kfold, batch_size=batch_size, shuffle=True)
for umpf_id in range(num_ump_f):
    print("current unmapped feature id: ", umpf_id)
    cur_model = Seq2seq_model(umpf_id, batch_size, 1, hidden_dim, seq_len, num_mp_f)
    cur_optimizer = optim.Adam(cur_model.parameters(), lr = lr)
    for i in range(num_epochs):
#         print(f"Epoch {i+1}\n-------------------------------")
        train(final_train_dataloader_2, cur_model, cur_optimizer, batch_size)
    
    torch.save(cur_model.state_dict(), "./each_umpf_model_param/EHR2/umpf_"+str(umpf_id+1)+"_EHR2.pth")

current unmapped feature id:  0
current unmapped feature id:  1
current unmapped feature id:  2
current unmapped feature id:  3
current unmapped feature id:  4
current unmapped feature id:  5
current unmapped feature id:  6
current unmapped feature id:  7
current unmapped feature id:  8
current unmapped feature id:  9
current unmapped feature id:  10
current unmapped feature id:  11
current unmapped feature id:  12
current unmapped feature id:  13
current unmapped feature id:  14


### e. Test the final trained model's performance on 2 EHRs  
* using dataset:  dataset_1_final_test & dataset_2_final_test

In [92]:
models_list_ump_EHR1 = []
models_list_ump_EHR2 = []
for i in range(1, 16):
    cur_model_1_pth = "./each_umpf_model_param/EHR1/umpf_" + str(i) + "_EHR1.pth"
    cur_model_2_pth = "./each_umpf_model_param/EHR2/umpf_" + str(i) + "_EHR2.pth"
    
    cur_model_1 = Seq2seq_model(i-1, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
    cur_model_1.load_state_dict(torch.load(cur_model_1_pth))
    
    cur_model_2 = Seq2seq_model(i-1, batch_size, input_dim, hidden_dim, seq_len, num_mp_f)
    cur_model_2.load_state_dict(torch.load(cur_model_2_pth))
    
    models_list_ump_EHR1.append(cur_model_1)
    models_list_ump_EHR2.append(cur_model_2)

In [57]:
dataloader_1_test = DataLoader(dataset_1_final_test, batch_size=10)
dataloader_2_test = DataLoader(dataset_2_final_test, batch_size=10)

In [67]:
len_dl_1 = len(dataloader_1_test)
len_dl_2 = len(dataloader_2_test)


In [93]:
num_ump_f_1 = 15
num_ump_f_2 = 15
avg_cos_sim_matrix = np.zeros((num_ump_f_1, num_ump_f_2))
for ump_i_1 in range(15):
    cur_model_1 = models_list_ump_EHR1[ump_i_1]
    cur_model_1.eval()
    cur_umpf_1_to_all_umpf_2 = np.zeros((len_dl_1, num_ump_f_2))
    for index, (data1, data2) in enumerate(zip(dataloader_1_test, dataloader_2_test)):
        pred_map_1 = cur_model_1(data1)["predicts"] 
        pred_map_1_asarray = pred_map_1.detach().numpy() # (10, 120)
        for ump_i_2 in range(15):
            cur_model_2 = models_list_ump_EHR2[ump_i_2]
            cur_model_2.eval()
            pred_map_2 = cur_model_2(data2)["predicts"]
            pred_map_2_asarray = pred_map_2.detach().numpy() # (10, 120)
            cur_cosine_sim = np.sum(pred_map_1_asarray*pred_map_2_asarray, axis=1) / (norm(pred_map_1_asarray, axis=1)*norm(pred_map_2_asarray, axis=1))
            cur_umpf_1_to_all_umpf_2[index][ump_i_2] = np.average(cur_cosine_sim)
    avg_cos_sim_matrix[ump_i_1, :] = np.average(cur_umpf_1_to_all_umpf_2, axis=0)

    
print(avg_cos_sim_matrix)
 

        

[[ 3.74261856e-02  2.39768243e-02  3.25501652e-02  2.52910724e-02
  -2.06761370e-03  3.72644125e-02 -4.39876936e-03  2.06935651e-02
   8.66800412e-03  3.09546529e-02 -3.91203036e-02  3.71582729e-02
   2.58954518e-02  4.81017568e-02  1.02139264e-05]
 [ 1.74765666e-02  8.55164400e-03 -1.47844531e-02  2.74632012e-02
   3.33786751e-02 -1.14453827e-02 -4.23046756e-02  1.85523012e-02
   4.34630819e-03  5.42731808e-03 -5.91968254e-02 -1.01243979e-02
  -4.48608652e-02  3.05853141e-04 -6.34039233e-02]
 [ 2.28576269e-02  2.04307055e-02 -4.77591167e-03  4.57326017e-02
   9.86208860e-04  1.97132845e-02 -3.95645911e-02  5.90214178e-02
   3.39519310e-02  1.30725086e-02 -5.42575976e-02 -3.04407598e-02
   5.14139559e-02  2.44664263e-03  2.06064633e-02]
 [-2.36018157e-02  3.59492260e-02 -7.70573418e-03  4.44698165e-02
  -2.46900562e-02  1.25560965e-03  4.01685046e-02  3.04423901e-02
  -4.66275722e-03  4.87156400e-03 -7.50882515e-02 -6.25320591e-03
   4.26782624e-02  2.15550785e-02 -3.55003559e-02]
 [-4

# The predicted matched features for each unmapped feature in EHR1

In [94]:
pred_matched_for_umf1 = avg_cos_sim_matrix.argmax(axis = 1)
pred_matched_for_umf1

array([13,  4,  7,  3,  1,  1, 14,  2, 10,  3, 10,  4,  4, 14, 10])

# The actual matched feature for each unmapped feature in EHR1

In [95]:
actual_matched_for_umf1 = p.argmax(axis=1)
actual_matched_for_umf1

array([ 2,  5, 13,  4, 11,  9,  6, 14,  1,  0, 12,  3, 10,  8,  7])

In [96]:
predicted_perf = (pred_matched_for_umf1 == actual_matched_for_umf1)

In [97]:
predicted_perf

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])